In [ ]:
from __future__ import annotations

from monty.os.path import zpath
from monty.serialization import loadfn
import os
from pathlib import Path

from pymatgen.io.validation.validation import VaspValidator

from pymatgen.io.vasp import PotcarSingle, Potcar

For copyright reasons, the POTCAR for these calculations cannot be distributed with this file, but its summary stats can.

If you have the POTCAR resources set up in pymatgen, you can regenerate the POTCARs used here by enabling `regen_potcars`

In [ ]:
regen_potcars = True

def get_potcar_from_spec(potcar_spec : dict) -> Potcar | None:
    
    for functional in PotcarSingle._potcar_summary_stats:

        potcar = Potcar()
        matched = [False for _ in range(len(potcar_spec))]
        for ispec, spec in enumerate(potcar_spec):
            titel = spec.get("titel","")
            titel_no_spc = titel.replace(" ","")
            symbol = titel.split(" ")[1].strip()
                
            for stats in PotcarSingle._potcar_summary_stats[functional].get(titel_no_spc,[]):
                
                if PotcarSingle.compare_potcar_stats(spec["summary_stats"], stats):
                    potcar.append(PotcarSingle.from_symbol_and_functional(symbol=symbol, functional=functional))
                    matched[ispec] = True
                    break
                    
            if all(matched):
                return potcar
    
def check_calc(calc_dir : str | Path) -> VaspValidator:

    calc_dir = Path(calc_dir)
    potcar_filename = None
    if regen_potcars:
        potcar = get_potcar_from_spec(loadfn(calc_dir / "POTCAR.spec.gz"))
        if potcar:
            potcar_filename = calc_dir / "POTCAR.gz"
            potcar.write_file(potcar_filename)
    
    vasp_files = {
        k.lower().split(".")[0] : zpath(calc_dir / k) for k in (
            "INCAR","KPOINTS","POSCAR","POTCAR","OUTCAR", "vasprun.xml"
        )
    }
    
    valid_doc = VaspValidator.from_vasp_input(
        vasp_file_paths={
            k : v for k,v in vasp_files.items() if Path(v).exists()
        },
        check_potcar=(regen_potcars and potcar)
    )

    if potcar_filename and potcar:
        os.remove(potcar_filename)
        
    return valid_doc
    

An example of an MP-compatible r2SCAN static calculation for GaAs is located in the `MP_compliant` directory.

In [ ]:
mp_compliant_doc = check_calc("MP_compliant")
print(mp_compliant_doc.is_valid)

We also include `TaskDoc` objects generated with `atomate2`, the workflow software currently used by the Materials Project (MP) for high-throughput calculations. A `TaskDoc` is also the document schema for the MP `task` collection.

If you have `emmet-core` (this is the software used to build Materials Project data) or `atomate2` installed, you can load a `TaskDoc` like this:

In [ ]:
from emmet.core.tasks import TaskDoc
from pymatgen.io.validation.emmet_validation import ValidationDoc

compliant_task_doc = TaskDoc(
    **loadfn(os.path.join("MP_compliant","MP_compatible_GaAs_r2SCAN_static.json.gz"))
)
mp_compliant_doc = ValidationDoc.from_task_doc(compliant_task_doc)
print(mp_compliant_doc.valid)

An example of an MP incompatible r<sup>2</sup>SCAN static calculation for GaAs is located in the `MP_non_compliant` directory.

This calculation uses a lower ENCUT, ENAUG, and k-point density (larger KSPACING) than is permitted by the appropriate input set, `pymatgen.io.vasp.sets.MPScanStaticSet`.
These reasons are reflected transparently in the output reasons.

In [ ]:
mp_non_compliant_doc = check_calc("MP_non_compliant")
print(mp_non_compliant_doc.is_valid)
for reason in mp_non_compliant_doc.reasons:
    print(reason)